# Discovery Consultant SQL Workshop SOLUTION

Welcome to the hands-on portion of this workshop! In this notebook, you will walk through writing basic queries using the **movies** dataset.

If you're not familiar with Jupyter Notebooks, press **shift + enter** to run cells. If you accidentally click on one of the text cells (Markdown cells), just run it and it will look normal again. When you re-open the notebook, you need to re-run all the cells (go to Cell tab at the top to do this).

First, run the cells below to set up the notebook.

In [2]:
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite://', echo=False)

## Movies Dataset
First we need to read in our Movies dataset .csv file into a pandas dataframe. This way, it is easier to create a sqlite engine to run SQL statements on our dataset.

In [4]:
movies = pd.read_csv('movie_metadata.csv')

In [5]:
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


To find all the column names, we can run `movies.columns`:

In [6]:
movies.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

The `to_sql` method writes records stored in a dataframe to a SQL database.

In [7]:
movies.to_sql('movies', con=engine)

5043

## 1. Examples
What do you think these example SQL statements will output? Note that when we run SQL queries in this notebook, they must be written in the format `engine.execute("...").fetchall()` in order to run correctly. Additionally, the result will be a list.

**Question 1.1** This query uses `DISTINCT` and `WHERE`.

In [8]:
query = """
        SELECT distinct(actor_2_name)
        FROM movies
        WHERE imdb_score > 8.5
        """

pd.read_sql(query, engine).head()

,actor_2_name
0,Heath Ledger
1,Anne Hathaway
2,Tom Hardy
3,Orlando Bloom
4,Billy Boyd


**Question 1.2** This query uses `LIKE`.

In [9]:
query = """
        SELECT movie_title 
        FROM movies 
        WHERE genres LIKE '%Thriller%'
        """
pd.read_sql(query, engine).head()

,movie_title
0,Spectre
1,The Dark Knight Rises
2,Battleship
3,Jurassic World
4,Skyfall


**Question 1.3** This query uses `GROUP BY`, `ORDER BY`, and `LIMIT`.

In [10]:
query = """
        SELECT country, avg(duration) 
        FROM movies 
        GROUP BY country 
        ORDER BY avg(duration) 
        DESC LIMIT 10
        """
pd.read_sql(query, engine).head()

,country,avg(duration)
0,Iceland,230.333333
1,Libya,156.000000
2,West Germany,155.666667
3,Thailand,142.800000
4,Slovakia,141.000000


## 2. Your turn!
**Question 2.1** Show the titles of all the movies that have a content rating of R and durations under 100.

*Note:* Remember that you can call `movies.columns` to refer back to all the column names in the dataframe.

In [11]:
# engine.execute("SELECT ... FROM ... WHERE ...").fetchall()
query = """
        SELECT movie_title 
        FROM movies 
        WHERE content_rating = 'R' AND duration < 100
        """
pd.read_sql(query, engine).head()

,movie_title
0,The Campaign
1,Dredd
2,Swordfish
3,Soldier
4,Snake Eyes


**Question 2.2** Show the titles and the duration of all the movies that have Action as a genre. 

*Hint:* Note that a movie can have multiple genres, so you might want to check what the `genres` column looks like first.

In [12]:
# engine.execute("SELECT ... FROM movies WHERE ... LIKE ...").fetchall()
query = """
        SELECT movie_title, duration 
        FROM movies 
        WHERE genres LIKE '%Action%'
  """
pd.read_sql(query, engine).head()

,movie_title,duration
0,Avatar,178.0
1,Pirates of the Caribbean: At World's End,169.0
2,Spectre,148.0
3,The Dark Knight Rises,164.0
4,John Carter,132.0


**Question 2.3** Show the directors that earned the top 5 largest total gross sum.

In [13]:
# engine.execute("SELECT director_name, ... as sum FROM movies GROUP BY ... ORDER BY ... LIMIT ...").fetchall()
query = """
        SELECT director_name, SUM(gross) as sum 
        FROM movies 
        GROUP BY director_name 
        ORDER BY sum DESC 
        LIMIT 5;
        """
pd.read_sql(query, engine).head()

,director_name,sum
0,Steven Spielberg,4.114233e+09
1,Peter Jackson,2.592969e+09
2,Michael Bay,2.231243e+09
3,Tim Burton,2.071275e+09
4,Sam Raimi,2.049549e+09


**Question 2.4** Show the 5 countries that have the highest average IMDB scores.

In [14]:
# engine.execute("SELECT ... FROM movies GROUP BY ... ORDER BY ... LIMIT ...").fetchall()
query = """
        SELECT country, AVG(imdb_score) as avg_imdb 
        FROM movies 
        GROUP BY country 
        ORDER BY avg_imdb DESC 
        LIMIT 5;
        """
pd.read_sql(query, engine).head()

,country,avg_imdb
0,Kyrgyzstan,8.7
1,Libya,8.4
2,United Arab Emirates,8.2
3,Soviet Union,8.1
4,Egypt,8.1


## 3. Harder SQL

In [15]:
# run to look at as a reference
movies.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


---
**Question 3.1** Of the movies that have an `imdb_score` lower than 5, find the number of `English` movies `made in the U.S` or `made in the U.K` .

In [16]:
# engine.execute(""" SELECT ... FROM ... WHERE ...""").fetchall()

# Using a subquery:
# engine.execute(""" SELECT COUNT(*) FROM (SELECT * FROM movies WHERE imdb_score > 8) WHERE language = 'English' AND country='USA'
#                 """).fetchall()

# Without a subquery:
query = """
        SELECT COUNT(movie_title) 
        FROM movies 
        WHERE imdb_score < 5 AND language = 'English' AND (country = 'USA' OR country = 'UK')
        """
pd.read_sql(query, engine).head()

,COUNT(movie_title)
0,412


---
**3.2** Find all the directors with an average `imdb_score` of less than 5 across all their movies, with the worst director on top.

In [18]:
query = """
        SELECT director_name, AVG(imdb_score) 
        FROM movies 
        GROUP BY director_name
        HAVING AVG(imdb_score) < 5
        ORDER BY AVG(imdb_score) ASC;
        """
pd.read_sql(query, engine).head()

,director_name,AVG(imdb_score)
0,Lawrence Kasanoff,1.7
1,A. Raven Cruz,1.9
2,Frédéric Auburtin,2.0
3,Vondie Curtis-Hall,2.1
4,Georgia Hilton,2.2


---
**3.3** Find the `content_rating`, the `COUNT`, and the `AVG` movie Facebook likes of the number of movies of that type `ORDER BY` least popular on average based on Facebook likes.

In [19]:
query = """
        SELECT content_rating, COUNT(movie_title), AVG(movie_facebook_likes)
        FROM movies
        GROUP BY content_rating
        ORDER BY AVG(movie_facebook_likes) ASC;
        """
pd.read_sql(query, engine).head()

,content_rating,COUNT(movie_title),AVG(movie_facebook_likes)
0,GP,6,25.000000
1,M,5,122.800000
2,TV-PG,13,225.846154
3,TV-Y,1,301.000000
4,TV-G,10,336.000000


---
**3.4** Find two different ways to get the name and length of the longest movie in the dataset. The length of the moving must not be missing (not none).

*Hint*: One potential solution involves using subqueries.

In [20]:
# a few possible solutions:

#query = """
#        SELECT movie_title, duration
#        FROM movies
#        WHERE duration >= 0
#        ORDER BY duration
#        LIMIT 1
#        """

# query = """
#         SELECT GROUP_CONCAT(movie_title, ', ') AS movie_title, duration
#         FROM movies
#         GROUP BY duration
#         HAVING duration = MIN(duration)
#         """

# advanced; uses a subquery!
query = """
        SELECT movie_title, duration
        FROM movies
        WHERE duration = (SELECT MAX(duration)
                          FROM movies)
         """

pd.read_sql(query, engine).head()

,movie_title,duration
0,Trapped,511.0


---
**3.5** Find the movie_titles that start with the letter 'K' that were directed by directors with an avg `imdb_score` rating of over 5 (if a director has an average imdb score above 5, find all of the movies that were directed by that director that start with the letter 'K').

In [32]:
query = """
        SELECT director_name, movie_title
        FROM movies
        WHERE movie_title LIKE 'K%'
        AND director_name IN (SELECT director_name 
                     FROM movies 
                     GROUP BY director_name 
                     HAVING AVG(imdb_score) > 5)
        """

pd.read_sql(query, engine)

,director_name,movie_title
0,Peter Jackson,King Kong
1,Jennifer Yuh Nelson,Kung Fu Panda 2
2,Alessandro Carloni,Kung Fu Panda 3
3,Mark Osborne,Kung Fu Panda
4,James Mangold,Knight and Day
5,Ridley Scott,Kingdom of Heaven
6,Kathryn Bigelow,K-19: The Widowmaker
7,Robert Luketic,Killers
8,Gary McKendry,Killer Elite
9,Quentin Tarantino,Kill Bill: Vol. 1


---

Thoughtful Question: Why might we have an issue with using ORDER BY and LIMIT 1 to find the maximum or minimum?  

Answer: In the case that we have more than one maximum or minimum, LIMIT 1 would prevent us from seeing it. As you write more SQL, you'll see these edge cases more clearly, so be very clear what the question is asking, and what your query is returning!

**CHALLENGE:** Open-ended questions to try to find the answer to!

1. How can we rate the movies in terms of popularity? One way to approach this is to assume that IMDB's score is not a holistic view of a given movie, and that we can use Facebook likes as a good measurement of popularity. Try weighting each column, ex:  

| Column         | Weight |
|----------------|--------|
| IMDB Score     | 40%    |
| Actor 1 Likes  | 10%    |
| Actor 2 Likes  | 10%    |
| Actor 3 Likes  | 10%    |
| Director Likes | 30%    |

2. Which country is the most popular in terms of movies they produced? Try using the table you created from question 1's query to answer this question. Set a variable equal to the output after calling `pd.read_sql()`, and then use that variable name in your FROM statement.